In [1]:
import sys
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.activations import sigmoid
from keras.activations import relu
from keras.activations import tanh
from keras.losses import MSE
from keras.losses import mean_squared_error
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.metrics import binary_accuracy
from tensorflow.compat.v1.graph_util import convert_variables_to_constants
import os

Using TensorFlow backend.


In [2]:
sess=tf.Session()
K.set_session(sess)

In [3]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [4]:
model = Sequential()
model.add(Dense(4, input_dim=2, activation='sigmoid'))
# model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer=SGD(lr=1), metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.1), metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
epochs = 1000
model.fit(training_data, target_data, verbose=0, epochs=epochs)

Instructions for updating:
Use tf.cast instead.


In [6]:
print ("loss:", model.evaluate(x=training_data, y=target_data, verbose=0))
print ("")
print (model.predict(training_data))
print ("")
print (model.predict(training_data).round())
print ("")
print ("w0:", model.layers[0].get_weights()[0])
print ("b0:", model.layers[0].get_weights()[1])
print ("w1:", model.layers[1].get_weights()[0])
print ("b1:", model.layers[1].get_weights()[1])



loss: [0.007118963170796633, 1.0]

[[0.08846797]
 [0.91719484]
 [0.915924  ]
 [0.08199881]]

[[0.]
 [1.]
 [1.]
 [0.]]

w0: [[ 1.8241819   4.8605204  -4.2114124   0.06415245]
 [-1.610837   -5.085408    3.8166788  -0.87375915]]
b0: [ 1.2008116  -2.6972873  -2.1484764   0.25374064]
w1: [[-2.2196918]
 [ 6.8599257]
 [ 5.529104 ]
 [-1.277896 ]]
b1: [-0.91743577]


In [7]:
print (model.predict(x=np.array([[1,0]]))[0][0])

0.9159239


## Save
### freeze model and save as Tensorflow 

In [8]:
### freeze model and save as Tensorflow 

freeze_var_names = list(set(v.op.name for v in tf.global_variables()))
output_names = [model.outputs[0].name.split(":")[0]]

frozen_graph = convert_variables_to_constants(sess,
                                              sess.graph.as_graph_def(),
                                              output_names,
                                              freeze_var_names)

tf.train.write_graph(frozen_graph, "models", "xor.pb", as_text=False)
print ("xor.pb written")
print (model.inputs[0].name)
print (model.outputs[-1].name)



Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.
xor.pb written
dense_1_input:0
dense_2/Sigmoid:0


### save as a keras h5 model

In [9]:
model.save("models/xor.h5")